In [1]:
from time import perf_counter

import cantera as ct
import torch
from torch.autograd.functional import jacobian as jacobian

import reactorch as rt

cpu = torch.device('cpu')

cuda = torch.device('cuda:0')

device = cuda

mech_yaml = '../data/gri30.yaml'
composition = "CH4:0.5, O2:1.0, N2:3.76"

sol = rt.Solution(mech_yaml=mech_yaml, device=device)

gas = sol.gas
gas.TPX = 950, 20 * ct.one_atm, composition

r = ct.IdealGasReactor(gas)
sim = ct.ReactorNet([r])

time = 0.0
t_end = 10
idt = 0
states = ct.SolutionArray(gas, extra=['t'])
T0 = gas.T

print('%10s %10s %10s %14s' % ('t [s]', 'T [K]', 'P [atm]', 'u [J/kg]'))

while sim.time < t_end:

    sim.step()

    states.append(r.thermo.state, t=time)

    if r.thermo.T > T0 + 600 and idt < 1e-10:
        idt = sim.time

    if idt > 1e-10 and sim.time > 4 * idt:
        break

print('%10.3e %10.3f %10.3f %14.6e' % (sim.time,
                                       r.T,
                                       r.thermo.P / ct.one_atm,
                                       r.thermo.u))

print('idt = {:.2e} [s] number of points {}'.format(idt, states.t.shape[0]))

     t [s]      T [K]    P [atm]       u [J/kg]
 4.807e-01   2943.750     63.269   2.355070e+05
idt = 1.01e-01 [s] number of points 3233


In [2]:
TP = torch.stack((torch.Tensor(states.T), torch.Tensor(states.P)), dim=-1)
Y = torch.Tensor(states.Y)
TPY = torch.cat([TP, Y], dim=-1).to(device)

TPY.requires_grad = True

In [3]:
%timeit sol.set_states(TPY)

61.2 ms ± 1.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%timeit sol.forward_rate_constants_func()

58.2 ms ± 38.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


It seems that the bottleneck is the function of `forward_rate_constants_func()`

In [5]:
%timeit sol.equilibrium_constants_func()

942 µs ± 706 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
%timeit sol.reverse_rate_constants_func()

86 µs ± 20.2 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
%timeit sol.wdot_func()

1.27 ms ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
# Test if the AD works properly

%time

TPY_grad = torch.autograd.grad(outputs=sol.wdot.sum(),
                               inputs=TPY,
                               retain_graph=True,
                               create_graph=True,
                               allow_unused=True)[0]

print(TPY_grad.shape)

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 11.2 µs
torch.Size([3233, 55])
